In [30]:
import bilby
import numpy as np 
import time

C = 299792458.
G = 6.67408*1e-11
Mo = 1.989*1e30
Mpc = 3.086*1e22

In [10]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [31]:
import hphc

In [21]:
t1 = time.time()

zz = hphc.htilde(0.4, 0.1, 200.0*Mpc ,np.arange(20.,200.,1., dtype='float64'), 20.0, 0.0, 1.2, 1180002601.0, 45.0*Mo, 0.24950617283950619, 97.68922304308887, 3.9780000000000004e+30, 100)

t2 = time.time()
t = t2 - t1
print("%.20f" % t)

0.37585544586181640625


In [32]:
from eccn.functions import eccentric_waveform

In [34]:
eccentric_waveform([50.],19.564163812778446,0.9148936170212766,0.1,200.0,0.4,0.1,1.2,1180002601.0,45,5.73)

{'plus': array([3.41493752e-23+1.90422629e-23j]),
 'cross': array([-1.8898067e-23+3.40504773e-23j])}

In [20]:
(C**3)/( G*(23.499999999999996+21.499999999999993)*Mo*np.pi*6**(3/2) )

97.6892230430889

In [17]:
def eccentric_waveform(frequency_array_, chirp_mass, mass_ratio, eccentricity, luminosity_distance, theta_jn, psi, phase, geocent_time, ra, dec):
    
    mass_1 = (chirp_mass*(1+mass_ratio)**(1/5))/mass_ratio**(3/5)
    mass_2 = chirp_mass*mass_ratio**(2/5)*(1+mass_ratio)**(1/5)
    luminosity_distance = luminosity_distance*Mpc
    total_mass = (mass_1+mass_2)*Mo
    symmetric_mass_ratio = (mass_1*mass_2)/((mass_1+mass_2)**2)
    lso_f = (C**3)/( G*(mass_1+mass_2)*Mo*np.pi*6**(3/2) )
    mass_diff = (mass_1-mass_2)*Mo
    f_min = 20.
    f_max = lso_f
    
    foo = np.array(frequency_array_,dtype='float64')

    h_plus,h_cross = hphc.htilde(theta_jn, psi, luminosity_distance , foo, f_min, eccentricity, phase, 0., total_mass, symmetric_mass_ratio, lso_f, mass_diff, f_max)

    return {'plus': h_plus, 'cross': h_cross}

In [2]:
import eccn.functions as fn

In [18]:
eccentric_waveform([50],19.564163812778446,0.9148936170212766,0.1,200.0,0.4,0.1,1.2,1180002601.0,45,5.73)

{'plus': array([-6.93275566e-25+3.90935893e-23j]),
 'cross': array([-3.89383983e-23-6.12731339e-25j])}

In [13]:
t1 = time.time()

zz = fn.eccentric_waveform(np.arange(20.,200.,1.),19.564163812778446,0.9148936170212766,0.1,200.0,0.4,0.1,1.2,1180002601.0,45,5.73)

t2 = time.time()
t = t2 - t1
print("%.20f" % t)

0.32152223587036132812


In [14]:
zz

{'plus': array([-6.48035476e-23-7.97603251e-23j,
         9.72247779e-23+1.03899103e-22j,
         5.21409322e-23-1.31194195e-22j,
        -6.03895868e-23-5.63600159e-23j,
        -1.19817950e-22-1.16516712e-23j,
        -7.41523256e-23-2.57947040e-23j,
        -2.14245329e-23-6.58055903e-23j,
         6.95337343e-23-4.53116876e-23j,
         3.17764753e-23+5.98848863e-23j,
        -5.51425953e-23+2.82954372e-23j,
         1.21419285e-23-7.25341825e-23j,
         2.01816873e-23+7.98046717e-23j,
        -3.13613420e-23-4.97392957e-23j,
         6.38980741e-23+4.38563434e-23j,
        -3.30582796e-23-5.65094325e-23j,
         4.23221888e-23+4.85856970e-23j,
        -5.21815790e-24-5.54364807e-23j,
        -1.12865540e-23+6.90537471e-23j,
         3.72910611e-23-3.19914210e-23j,
        -6.02225503e-23-1.37492255e-23j,
         4.08257764e-23+4.21828696e-23j,
         8.14987032e-24-4.29028661e-23j,
        -4.83555263e-23+8.39725552e-24j,
         4.00935747e-23+3.95603647e-23j,
        

### cython speed up 2.7x
##### worth it !?

In [14]:
t1 = time.time()

zz = bilby.gw.source.lal_eccentric_binary_black_hole_no_spins( np.arange(20.,200.,1.) ,25.0,20.0,0.1,200.0,0.4,0.1)

t2 = time.time()
t = t2 - t1
print("%.20f" % t)

0.00252079963684082031


In [32]:
t1 = time.time()

zz = bilby.gw.source.lal_binary_black_hole( np.arange(20.,200.,0.1) ,25.0,20.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)

t2 = time.time()
t = t2 - t1
print("%.20f" % t)

0.00364470481872558594


In [17]:
bilby.gw.source.lal_binary_black_hole

<function bilby.gw.source.lal_binary_black_hole(frequency_array, mass_1, mass_2, luminosity_distance, a_1, tilt_1, phi_12, a_2, tilt_2, phi_jl, theta_jn, phase, **kwargs)>